In [46]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import Markdown as md
from ipynb.fs.full import functions
import pandas as pd

InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [47]:
ticker_prices_filename = 'ticker_prices_cleaned.xlsx'
ticker_prices_excel_book = pd.read_excel(io=ticker_prices_filename, header=[0,1], index_col=0, sheet_name=None)

market_prices_filename = 'market_prices.xlsx'
market_prices_excel_book = pd.read_excel(io=market_prices_filename, index_col=0, sheet_name=None)

In [49]:
period_column_name = 'period'
Close_column_name = 'Close'
price_column_name = 'price'
Volume_column_name = 'Volume'
volume_column_name = 'volume'
first_index = -11
market_suffix = '.JK'

tickers_output_sheet_list = []
market_output_sheet_list = []

for sheet_name in ticker_prices_excel_book.keys():
    tickers_data_frame = ticker_prices_excel_book[sheet_name].copy()
    tickers_number_of_rows = len(tickers_data_frame.index)
    tickers_data_frame[period_column_name] = list(range(first_index, first_index+tickers_number_of_rows))
    tickers_data_frame = tickers_data_frame.copy().set_index(period_column_name)
    tickers_data_frame = tickers_data_frame.copy().rename(columns={Close_column_name: price_column_name, Volume_column_name: volume_column_name}, level=0)
    tickers_data_frame.columns = pd.MultiIndex.from_tuples([(i[0], i[1].replace(market_suffix, '')) for i in tickers_data_frame.columns])
    
    tickers_output_sheet_list.append({
        'sheet_name': sheet_name,
        'data_frame': tickers_data_frame,
    })

    market_data_frame = market_prices_excel_book[sheet_name].copy()
    market_number_of_rows = len(market_data_frame.index)
    market_data_frame[period_column_name] = list(range(first_index, first_index+market_number_of_rows))
    market_data_frame = market_data_frame.copy().set_index(period_column_name)
    market_data_frame = market_data_frame.copy().rename(columns={Close_column_name: price_column_name, Volume_column_name: volume_column_name}, level=0)

    market_output_sheet_list.append({
        'sheet_name': sheet_name,
        'data_frame': market_data_frame,
    })


In [50]:
with pd.ExcelWriter('ticker_prices_transformed.xlsx') as excel_writer:
    for i in tickers_output_sheet_list:
        sheet_name = i['sheet_name']
        df = i['data_frame']
        df.to_excel(excel_writer=excel_writer, sheet_name=sheet_name)

with pd.ExcelWriter('market_prices_transformed.xlsx') as excel_writer:
    for i in market_output_sheet_list:
        sheet_name = i['sheet_name']
        df = i['data_frame']
        df.to_excel(excel_writer=excel_writer, sheet_name=sheet_name)
